In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import warnings
from langchain_core._api import LangChainBetaWarning
from rag_engine.retriever import build_retriever
from rag_engine.chains import get_rag_chain
from rag_engine.memory import ConversationMemory

warnings.filterwarnings("ignore", category=LangChainBetaWarning)
load_dotenv()
retriever = build_retriever()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
rag_chain = get_rag_chain(llm, retriever)

# Conversation memory: remembers past Q&A; summarizes when large to avoid explosion
memory = ConversationMemory(
    llm,
    max_messages_before_summary=10,  # summarize when more than 10 turns
    recent_turns_to_keep=4,          # keep last 4 turns in full after summarizing
)

In [5]:
# Set your question here; run this cell to get an answer (with conversation memory)
question = "What is Assistive and Rehabilitation Robotics Laboratory?"
chat_history = memory.get_chat_history_for_prompt()

chunks = []
for chunk in rag_chain.stream({"question": question, "chat_history": chat_history}):
    chunks.append(chunk)
    print(chunk, end="", flush=True)
full_response = "".join(chunks)
memory.add_turn(question, full_response)

The **Assistive and Rehabilitation Robotics Laboratory** is a research facility at Northwestern University that focuses on advancing human ability through robotics autonomy. The lab aims to ease the burden of controlling assistive machines, particularly for individuals with motor impairments. 

### Key Details:
- **Director**: Brenna Argall
- **Department**: Computer Science
- **Lab Description**: The lab develops customized robotics autonomy and intelligence to help bridge the gap left by sensory, motor, or cognitive impairments in users of assistive machines. It explores how to share control between the robot and the human user and adapts these control formulations over time as the human changes. The lab works with various hardware platforms, including smart wheelchairs and assistive robotic arms.
- **Website**: [Assistive and Rehabilitation Robotics Laboratory](https://www.sralab.org/research/labs/assistive-rehabilitation-robotics-lab)

The lab's research is directed towards increas

In [ ]:
# Run this cell to clear all conversation memory and start fresh
memory.clear()
print("Conversation memory cleared.")